In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import KBinsDiscretizer
from scikit_weak.feature_selection import RoughSetSelector
from scikit_weak.classification import SupersetKNeighborsClassifier, DELIN, RRLClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import numpy.random as random
from sklearn.metrics import accuracy_score

In [2]:
X, y = load_iris(return_X_y=True)
est = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='kmeans')

X_discr = est.fit_transform(X)
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [3]:
np.random.seed(0)

noise = 0.10
target = []
for t in y:
    t_temp = [t]
    classes = np.unique(y)
    for c in classes:
        if random.uniform(size=1) <= noise and c != t:
            t_temp.append(c)
    target.append(t_temp)

target = np.array(target)
target

<ipython-input-3-296739688c8b>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  target = np.array(target)


array([list([0]), list([0]), list([0]), list([0]), list([0, 2]),
       list([0, 1]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0, 1]), list([0]), list([0]), list([0, 1]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0, 1]), list([0]), list([0]), list([0]),
       list([0]), list([0, 1]), list([0]), list([0]), list([0]),
       list([0]), list([0, 1]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([0]), list([0]), list([0]), list([0]),
       list([0]), list([0]), list([1]), list([1]), list([1]),
       list([1, 0]), list([1]), list([1]), list([1]), list([1, 0, 2]),
       list([1]), list([1, 2]), list([1]), list([1]), list([1]),
       list([1]), list([1]), list([1]), list([1, 0]), list([1]),
       list([1, 0]), list([1]), list([1]), list([1]), list([1]),
       list([1]), list([1]), list([1]), list([1]), list([1, 

In [4]:
rs = RoughSetSelector(method='approximate', epsilon = 0.01, discrete=True)
rs.fit(X_discr, target)
print(rs.reducts_)

[0, 2, 3]


In [5]:
rs = RoughSetSelector(method='brute', epsilon = 0.01, discrete=True)
rs.fit(X_discr, target)
print(rs.reducts_)

[(0, 2, 3)]


In [6]:
rs = RoughSetSelector(method='brute', epsilon = 0.01, discrete=False,
                      neighborhood='delta', radius=0.1)
rs.fit(X, target)
print(rs.reducts_)

[(0, 1, 3), (1, 2, 3)]


In [7]:
rs = RoughSetSelector(method='brute', epsilon = 0.01, discrete=False,
                      neighborhood='nearest', n_neighbors=10)
rs.fit(X, target)
print(rs.reducts_)

[(2, 3)]


In [8]:
clf = RRLClassifier(estimator=DecisionTreeClassifier, missing=False, resample=True)
clf.fit(X, target)
accuracy_score(y, clf.predict(X))

0.9933333333333333

In [9]:
clf = SupersetKNeighborsClassifier()
clf.fit(X, target)
accuracy_score(y, clf.predict(X))

0.96

In [10]:
clf = DELIN()
clf.fit(X, target)
accuracy_score(y, clf.predict(X))

0.9733333333333334